In [59]:
import pandas as pd

I downloaded the current member list from USA Fencing's website on 7/19/22: https://member.usafencing.org/search/members

We will use the Python Pandas package to parse the provided .csv file and estimate how many fencers would be impacted by the loss of Div II/Div III NACs in the 2022/2023 season.

First, we will check that the .csv file was imported as expected by the Pandas package.

In [60]:
raw = pd.read_csv('members.csv')
raw.head()

,Last Name,First Name,Middle Name,Suffix,Nickname,Gender,Birthdate,Birthdate verified,Division,Section,...,Referee USA Rating Saber,Referee USA Year Saber,Referee FIE Rating Foil,Referee FIE Year Foil,Referee FIE Rating Epee,Referee FIE Year Epee,Referee FIE Rating Saber,Referee FIE Year Saber,Updated At,Last Modified
0,',Shevek,NaN,NaN,NaN,M,1979,No,Western Washington,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-04-12 08:13:28,2022-04-12 08:13:28
1,A,Yousef,NaN,NaN,NaN,M,2012,No,Western Washington,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-08-25 10:20:20,2021-08-25 10:20:20
2,A Goode,Gregory,NaN,NaN,NaN,M,1975,No,Oklahoma,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-09-10 04:45:13,2021-09-10 04:45:13
3,Aadhi,Hansika,NaN,NaN,NaN,F,2009,Yes,Michigan,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-07-19 04:15:52,2022-07-19 04:15:52
4,Aalberts,Cooper,NaN,NaN,NaN,M,2012,No,New Mexico,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-03-14 16:07:11,2022-03-14 16:07:11


Next, we will look for adults that are too old to fence Juniors, but too young to fence in Veteran events. Fencers in this age range are only eligible for Div I, Div 1A, Div II, or Div III events. The birth year guidance was taken for the 2021/2022 season from USA Fencing's website https://www.usafencing.org/age-classification-eligibility and adjusted to be for the upcoming season.

In [61]:
adults = raw[raw['Birthdate'].between(1983,2003)]

Next, we will look for active competitive memberships. A complete list of membership types included in the search is stored in the variable "possible_competitive_types" below. We have excluded coaching memberships, since these individuals may be only coaching, and we have excluded Parafencers, since they have dedicated NAC events.

In [62]:
possible_competitive_types = ['Collegiate Competitive', 'Competitive',
        'Life +CheckEd',
       'Life',
       'Competitive +CheckEd', 
       'Life Installment +CheckEd', 
       'Olympian Life +CheckEd',
       'International Competitive License +CheckEd',
       'Collegiate Competitive (Pending)',
       'Collegiate Competitive +CheckEd',
       'Collegiate Competitive +CheckEd (Pending)', 'Life Installment',
       'International Competitive License +CheckEd (Pending)',
       'Competitive +CheckEd (Pending)',
       'Olympian Life', 'Life Installment +CheckEd (Pending)',
       'Life +CheckEd (Pending)']
competitive_adults = adults[adults['Member Type'].isin(possible_competitive_types)]

Next, we look for members who could only fence in Div II/Div III events. That is, they do not have an A, B, or C in ANY weapon for the 2022/2023 season (C18 is allowed since this will expire for 2022/2023). Thus, we are not including any fencers who have a Div I eligible rating in one weapon, but would fence another weapon at a Div II/Div III level.

In [63]:
possible_div23_ratings = ['U', 'E20', 'D21', 'D19', 'E21', 'E22', 'E18', 'E19','D22',  'D18', 'C18', 'D20']
d23_comp_adults = competitive_adults[(competitive_adults.Saber.isin(possible_div23_ratings)) & (competitive_adults.Epee.isin(possible_div23_ratings) & competitive_adults.Foil.isin(possible_div23_ratings))]

Now, we subselect the columns for easier viewing and check the first few entries:

In [64]:
relevant_list = d23_comp_adults[['Last Name','First Name','Gender','Birthdate','Member Type','Saber','Epee','Foil']]
relevant_list.reset_index(inplace=True,drop=True)
relevant_list.head()

,Last Name,First Name,Gender,Birthdate,Member Type,Saber,Epee,Foil
0,Abbott,Autumn,F,2002,Competitive,U,U,U
1,Abdel-Tawab,Raghed,M,2000,Collegiate Competitive,U,U,U
2,Abdelaziz,Nadine,F,1994,Life +CheckEd,U,U,C18
3,Abdellahi,Aziz,M,1984,Competitive,E20,U,U
4,Abdullahi,Salma,F,2002,Competitive,U,U,U


And the last few entries:

In [65]:
relevant_list.tail()

,Last Name,First Name,Gender,Birthdate,Member Type,Saber,Epee,Foil
2418,Zug,Kiersten,F,1992,Competitive,U,U,U
2419,Zuiev,Xenia,F,1996,Competitive,U,U,U
2420,Zukofsky,Zachary,M,2003,Collegiate Competitive,U,E22,U
2421,Zunk,Patrick,M,1993,Competitive,U,U,U
2422,Zuvieta,Chelsea,F,2002,Collegiate Competitive,U,U,U


Now, let's find out how many men and women are affected:

In [66]:
relevant_list.Gender.value_counts()

M    1615
F     808
Name: Gender, dtype: int64